In [14]:
#import libraries and check versions
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
#import dataloader

print(tf.__version__)

1.15.0


In [18]:
num_epochs = 1000
train_size = 100
test_size = 100
#log_dir = "./logs

In [29]:
#clear logs from previous session runs
!rm -rf ./logs/

In [46]:
#load the data
data = pd.read_csv('/Users/vinitaboolchandani/workspaces/tensorflow-simulations/tf/outputs/output-12-10-10000-500/data_dump-500-20.out', sep="\t", header=None)

data.columns = ['rho', 'potential', 'temp']
data.head()
x = data.rho
y = data.drop('rho', axis=1)
#print(x)

#split the data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
y_train.shape
#print(x_train)

16

In [47]:
#define the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(x_train.shape[0], 1)),
    keras.layers.Dense(2, activation='relu'),
    keras.layers.Dense(2)
])

In [42]:
#compile the model
model.compile(optimizer='adam',
           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
              metrics=['accuracy']))

TypeError: __init__() got an unexpected keyword argument 'metrics'

In [2]:
#train the model
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(train_images, train_labels, epochs=num_epochs, callbacks=[tensorboard_callback]) #validation_data=(x_test, y_test), 

NameError: name 'model' is not defined

In [ ]:
#compare performance
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)